In [ ]:
import cv2
import os
import numpy as np
import tensorflow as tf

from random import shuffle
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras.layers import LSTM, Bidirectional, Conv2D, Dense, Flatten, MaxPooling2D, TimeDistributed, Reshape, MaxPooling3D
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ReduceLROnPlateau

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=0.0001)

cnn = Sequential()
cnn.add(Conv2D(64, (3, 3), activation='relu', input_shape=(10, IMG_SIZE, IMG_SIZE, 3), padding="same"))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling3D((1, 2,2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling3D((1, 2,2)))
cnn.add(Conv2D(64, (3, 3), activation='relu'))
cnn.add(MaxPooling3D((1, 2,2)))
cnn.add(Reshape((10, 6400)))

lstm_fw = LSTM(units=32, return_sequences=True)
lstm_bw = LSTM(units=32, go_backwards=True, return_sequences=True)

model = Sequential()
model.add(Bidirectional(lstm_fw, backward_layer=lstm_bw))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

opt = keras.optimizers.SGD(learning_rate=LR)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['accuracy'])

cnn.summary()

# Print model summary
model.summary()

model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_val, y_val), callbacks=[reduce_lr])
